In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install groq pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 4.2 MB/s eta 0:00:00


In [ ]:
# Cài đặt các thư viện cần thiết để chạy Local Model
!pip install -q torch transformers bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 18.9 MB/s eta 0:00:00


In [ ]:
!pip install -q -U bitsandbytes accelerate transformers

In [ ]:
!pip install -U bitsandbytes

In [ ]:
# ================= CELL 2: CHẠY LABELING (Local AI) =================
import pandas as pd
import json
import time
import os
import csv
import torch
import sys
from google.colab import drive
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# ================= CẤU HÌNH =================
INPUT_FILE_PATH = "/content/drive/MyDrive/Word_label_LLM/final_dataset_cleaned_v3_new.csv"
OUTPUT_FILE_PATH = "/content/drive/MyDrive/Word_label_LLM/output_labeling.csv"
MODEL_CACHE_DIR = "/content/drive/MyDrive/Colab_Models/Qwen2.5-7B"
MODEL_ID = "Qwen/Qwen2.5-7B-Instruct"

# 1. SETUP MÔI TRƯỜNG & MODEL
def setup_and_load():
    print("--- ĐANG KẾT NỐI GOOGLE DRIVE ---")
    drive.mount('/content/drive')

    if not os.path.exists(MODEL_CACHE_DIR):
        os.makedirs(MODEL_CACHE_DIR, exist_ok=True)

    print(f"--- ĐANG LOAD MODEL TỪ: {MODEL_CACHE_DIR} ---")

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16
    )

    try:
        tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, cache_dir=MODEL_CACHE_DIR, trust_remote_code=True)
        model = AutoModelForCausalLM.from_pretrained(
            MODEL_ID,
            cache_dir=MODEL_CACHE_DIR,
            quantization_config=bnb_config,
            device_map="auto",
            low_cpu_mem_usage=True,
            trust_remote_code=True
        )
        print("✅ LOAD MODEL THÀNH CÔNG!")
        return tokenizer, model
    except Exception as e:
        print(f"❌ LỖI LOAD MODEL: {e}")
        return None, None

tokenizer, model = setup_and_load()

# 2. HÀM HỎI AI VỚI PROMPT CHI TIẾT
def ask_local_ai(lyrics_text):
    # System Prompt: Định nghĩa vai trò
    system_prompt = "Bạn là chuyên gia ngôn ngữ và phân tích lời bài hát Việt Nam. Nhiệm vụ của bạn là trích xuất từ vựng chính xác theo mẫu."

    # User Prompt: Chứa các ví dụ cụ thể bạn yêu cầu
    user_prompt = f"""Hãy đọc đoạn Lyrics dưới đây và phân loại các từ vựng vào 4 nhóm theo định dạng JSON.

YÊU CẦU CHI TIẾT TỪNG NHÓM:

1. "hanviet":
   - Định nghĩa: Từ Hán-Việt, từ ngữ văn chương, từ láy gợi hình.
   - Ví dụ mẫu: chân thành, mộng mơ, ngây thơ, quý giá, viễn vông, phản bội, điên cuồng, tuyệt vọng...

2. "eng":
   - Định nghĩa: Từ tiếng Anh gốc hoặc Slang word (biến thể) mà rapper hay dùng.
   - Ví dụ mẫu: bae, finding, okay, phone, welcome, houhou, my baby, your smile, your body, can take my heart, halo, ya, lit, drip...

3. "nuoc_ngoai_phien_am":
   - Định nghĩa: Tiếng nước ngoài được phiên âm sang tiếng Việt hoặc viết kiểu Latin/Âm bồi.
   - Ví dụ mẫu: xa rang hê, ản tuê, ha chi ma, ha chi mán, xô, saranghaeyo, gomagoyo, watashi wa...

4. "ten_rieng":
   - Định nghĩa: Tên địa danh, tên ca sĩ, rapper, người nổi tiếng.
   - Ví dụ mẫu: hoài phúc, âu khải phong, vũ duy khánh, jombie, only t, tèo, lee yang, rubyn, mr.saker...

QUY TẮC BẮT BUỘC:
- Trả về duy nhất định dạng JSON.
- Các từ trong danh sách ngăn cách bằng DẤU PHẨY.
- Không có từ thì để chuỗi rỗng "".

Lyrics cần phân tích:
---------------------
{lyrics_text}
---------------------"""

    messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt}]

    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512,
        temperature=0.2,
        do_sample=True
    )

    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    try:
        start_idx = response.find('{')
        end_idx = response.rfind('}') + 1
        return json.loads(response[start_idx:end_idx]) if start_idx != -1 else None
    except:
        return None

# 3. CHƯƠNG TRÌNH CHÍNH
def main():
    if model is None: return

    if not os.path.exists(INPUT_FILE_PATH):
        print(f"❌ Không tìm thấy file input: {INPUT_FILE_PATH}")
        return

    # Đọc file (Hỗ trợ UTF-8 và UTF-16)
    try:
        df = pd.read_csv(INPUT_FILE_PATH, encoding='utf-8')
    except:
        df = pd.read_csv(INPUT_FILE_PATH, encoding='utf-16')

    # === SỬA LỖI TÊN CỘT ===
    # 1. Xóa khoảng trắng thừa ở đầu/cuối tên cột (VD: "lyrics " -> "lyrics")
    df.columns = df.columns.str.strip()
    # 2. Chuyển về chữ thường
    df.columns = df.columns.str.lower()

    # Tìm cột chứa chữ 'lyric' hoặc 'content'
    target_col = next((col for col in df.columns if 'lyrics' in col or 'content' in col), None)

    if not target_col:
        print(f"❌ Vẫn không tìm thấy cột Lyrics. Danh sách cột hiện có: {list(df.columns)}")
        return
    else:
        print(f"✅ Đã tìm thấy cột chứa lời bài hát: '{target_col}'")

    total_rows = len(df)

    # Checkpoint
    start_row = 0
    if os.path.exists(OUTPUT_FILE_PATH):
        with open(OUTPUT_FILE_PATH, 'r', encoding='utf-8-sig') as f:
            lines = sum(1 for line in f)
            start_row = max(0, lines - 1)
        print(f"[*] Tiếp tục chạy từ dòng: {start_row}/{total_rows}")
    else:
        print(f"[*] Tạo file mới tại: {OUTPUT_FILE_PATH}")

    with open(OUTPUT_FILE_PATH, mode='a', encoding='utf-8-sig', newline='') as f:
        # Cấu trúc cột output
        fieldnames = ['index_goc', 'lyrics', 'hanviet', 'eng', 'nuoc_ngoai_phien_am', 'ten_rieng']
        writer = csv.DictWriter(f, fieldnames=fieldnames)

        if start_row == 0: writer.writeheader()

        for i in range(start_row, total_rows):
            row = df.iloc[i]
            lyrics_text = str(row[target_col])

            print(f"[*] Dòng {i}/{total_rows}...", end="", flush=True)

            if len(lyrics_text) < 3 or lyrics_text.lower() == 'nan':
                writer.writerow({'index_goc': i, 'lyrics': 'SKIP', 'hanviet': '', 'eng': '', 'nuoc_ngoai_phien_am': '', 'ten_rieng': ''})
                print(" -> Bỏ qua")
                continue

            start_time = time.time()
            res = ask_local_ai(lyrics_text)

            if res:
                # Xử lý format: Thay dấu phẩy bằng xuống dòng (\n) để tạo list dọc trong Excel
                # Nếu bạn muốn giữ dấu phẩy thì xóa .replace(', ', '\n') đi
                writer.writerow({
                    'index_goc': i,
                    'lyrics': lyrics_text,
                    'hanviet': str(res.get('hanviet', '')).replace(', ', '\n').replace(',', '\n'),
                    'eng': str(res.get('eng', '')).replace(', ', '\n').replace(',', '\n'),
                    'nuoc_ngoai_phien_am': str(res.get('nuoc_ngoai_phien_am', '')).replace(', ', '\n').replace(',', '\n'),
                    'ten_rieng': str(res.get('ten_rieng', '')).replace(', ', '\n').replace(',', '\n')
                })
                f.flush()
                print(f" XONG ({time.time() - start_time:.1f}s)")
            else:
                writer.writerow({'index_goc': i, 'lyrics': lyrics_text, 'hanviet': 'ERROR', 'eng': '', 'nuoc_ngoai_phien_am': '', 'ten_rieng': ''})
                print(" LỖI")

            time.sleep(0.05)

    print("\n--- HOÀN THÀNH ---")

if __name__ == "__main__":
    main()

KeyboardInterrupt: 

**Cài đặt thư viện cần thiết chạy localAI**

In [ ]:
# ================= CELL 1: LOAD MODEL (CHẠY 1 LẦN) =================
import torch
import os
from google.colab import drive
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# CẤU HÌNH ĐƯỜNG DẪN MODEL
MODEL_CACHE_DIR = "/content/drive/MyDrive/Colab_Models/Qwen2.5-7B"
MODEL_ID = "Qwen/Qwen2.5-7B-Instruct"

def setup_and_load():
    print("--- ĐANG KẾT NỐI GOOGLE DRIVE ---")
    drive.mount('/content/drive')

    if not os.path.exists(MODEL_CACHE_DIR):
        os.makedirs(MODEL_CACHE_DIR, exist_ok=True)

    print(f"--- ĐANG LOAD MODEL TỪ: {MODEL_CACHE_DIR} ---")
    print("⏳ Vui lòng chờ 1-2 phút...")

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16
    )

    try:
        tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, cache_dir=MODEL_CACHE_DIR, trust_remote_code=True)
        model = AutoModelForCausalLM.from_pretrained(
            MODEL_ID,
            cache_dir=MODEL_CACHE_DIR,
            quantization_config=bnb_config,
            device_map="auto",
            low_cpu_mem_usage=True,
            trust_remote_code=True
        )
        print("✅ LOAD MODEL THÀNH CÔNG! Sẵn sàng chạy các Cell bên dưới.")
        return tokenizer, model
    except Exception as e:
        print(f"❌ LỖI LOAD MODEL: {e}")
        return None, None

# Biến toàn cục để các Cell sau dùng chung
tokenizer, model = setup_and_load()

RuntimeError: generic_type: cannot initialize type "RpcBackendOptions": an object with that name is already defined

**Cell này xử lý từ hanviet**

In [ ]:
# ================= CELL 3: HÁN VIỆT (ĐÃ FIX LỖI) =================
import pandas as pd
import json
import time
import os
import csv

INPUT_FILE_PATH = "/content/drive/MyDrive/Word_label_LLM/final_dataset_cleaned_v3_new.csv"
OUTPUT_FILE_PATH = "/content/drive/MyDrive/Word_label_LLM/output_part2_hanviet.csv"

def ask_group_2(lyrics_text):
    system_prompt = """
Bạn là chuyên gia ngôn ngữ học tiếng Việt, am hiểu sâu về từ Hán-Việt.
Bạn phân biệt rõ:
- Từ Hán-Việt
- Từ thuần Việt
- Từ vay mượn hiện đại

Bạn làm việc theo quy tắc nghiêm ngặt, không suy diễn.
"""

    user_prompt = f"""
NHIỆM VỤ:
Đọc lyrics và TRÍCH XUẤT TOÀN BỘ TỪ HÁN-VIỆT xuất hiện trong văn bản.

ĐỊNH NGHĨA TỪ HÁN-VIỆT:
- Từ có nguồn gốc Hán
- Thường là từ ghép 2–3 âm tiết
- Mang sắc thái trang trọng, cổ điển, văn viết
- Ví dụ hợp lệ:
  chân thành, cô quạnh, nhọc nhằn, bi ai, duyên phận, tương phùng,
  hồng nhan, bạc mệnh, định mệnh, nhân gian, trần thế, vô thường

LOẠI TRỪ (KHÔNG ĐƯỢC LẤY):
- Từ thuần Việt: yêu, nhớ, buồn, vui, khóc, cười, em, anh
- Khẩu ngữ / tiếng lóng: bad boy, chill, ok, baby
- Hư từ: thì, là, mà, với, đã, sẽ
- Tên riêng, địa danh, nghệ danh

YÊU CẦU OUTPUT:
- Chỉ trả về JSON
- Không giải thích
- Không thêm text bên ngoài
- Không lặp từ
- Giữ nguyên dạng từ trong lyrics

FORMAT:
{{
  "hanviet": "từ1, từ2, từ3"
}}

Nếu KHÔNG có từ Hán-Việt → để chuỗi rỗng "".

Lyrics:
{lyrics_text}
"""


    messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt}]

    # --- PHẦN SỬA LỖI ---
    if tokenizer.pad_token is None: tokenizer.pad_token = tokenizer.eos_token

    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        attention_mask=model_inputs.attention_mask, # <--- Fix lỗi
        pad_token_id=tokenizer.eos_token_id,        # <--- Fix lỗi
        max_new_tokens=512, temperature=0.2, do_sample=True
    )
    # --------------------

    response = tokenizer.batch_decode([out[len(inp):] for inp, out in zip(model_inputs.input_ids, generated_ids)], skip_special_tokens=True)[0]
    try:
        start = response.find('{'); end = response.rfind('}') + 1
        return json.loads(response[start:end]) if start != -1 else None
    except: return None

def main_group_2():
    if 'model' not in globals(): print("❌ Chưa load model!"); return
    try: df = pd.read_csv(INPUT_FILE_PATH, encoding='utf-8')
    except: df = pd.read_csv(INPUT_FILE_PATH, encoding='utf-16')

    df.columns = df.columns.str.strip().str.lower()
    target_col = next((col for col in df.columns if 'lyrics' in col or 'content' in col), None)

    start_row = 0
    if os.path.exists(OUTPUT_FILE_PATH):
        with open(OUTPUT_FILE_PATH, 'r', encoding='utf-8-sig') as f:
            start_row = max(0, sum(1 for line in f) - 1)
        print(f"[*] Tiếp tục từ dòng {start_row}")

    with open(OUTPUT_FILE_PATH, mode='a', encoding='utf-8-sig', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=['index_goc', 'lyrics', 'hanviet'])
        if start_row == 0: writer.writeheader()

        for i in range(start_row, len(df)):
            row = df.iloc[i]; lyrics = str(row[target_col])
            print(f"[*] Dòng {i} (HanViet)...", end="", flush=True)

            if len(lyrics) < 3 or lyrics.lower() == 'nan':
                writer.writerow({'index_goc': i, 'lyrics': 'SKIP', 'hanviet': ''})
                print(" -> Bỏ qua"); continue

            res = ask_group_2(lyrics)
            if res:
                writer.writerow({'index_goc': i, 'lyrics': lyrics, 'hanviet': str(res.get('hanviet', ''))})
                f.flush(); print(" ✅")
            else:
                writer.writerow({'index_goc': i, 'lyrics': lyrics, 'hanviet': 'ERROR'})
                print(" ❌")
            time.sleep(0.05)

if __name__ == "__main__": main_group_2()

**Tách cell ra xử lý , cell này xử lý eng, tên_riêng, phiên âm**

In [ ]:
# ================= CELL 2: ENG & TÊN RIÊNG (ĐÃ FIX LỖI) =================
import pandas as pd
import json
import time
import os
import csv

INPUT_FILE_PATH = "/content/drive/MyDrive/Word_label_LLM/final_dataset_cleaned_v3_new.csv"
OUTPUT_FILE_PATH = "/content/drive/MyDrive/Word_label_LLM/output_part1_eng_names.csv"

def ask_group_1(lyrics_text):
    system_prompt = """
Bạn là chuyên gia ngôn ngữ học và phân tích lyrics.
Bạn phân biệt chính xác:
- Tiếng Anh (kể cả slang rapper)
- Tên riêng
- Từ nước ngoài phiên âm sang tiếng Việt

Bạn làm việc theo quy tắc rõ ràng, không suy diễn, không đoán bừa.
"""

    user_prompt = f"""
NHIỆM VỤ:
Phân tích lyrics và trích xuất từ vựng theo 3 nhóm sau.
BỎ QUA toàn bộ từ Hán-Việt và từ thuần Việt.

====================
1️⃣ TIẾNG ANH (eng)
====================
BAO GỒM:
- Từ tiếng Anh trong từ điển:
  love, hate, money, baby, life, night
- Slang / rap culture:
  bae, chill, flex, homie, vibe, swag, lit, drip, opp, cap, no cap
- Từ viết thường hoặc viết hoa đều được

KHÔNG BAO GỒM:
- Tên riêng (đẩy sang ten_rieng)
- Phiên âm tiếng Anh bằng chữ Việt
- Từ Việt viết sai chính tả

====================
2️⃣ PHIÊN ÂM NƯỚC NGOÀI (nuoc_ngoai_phien_am)
====================
ĐỊNH NGHĨA:
- Từ nước ngoài được VIỆT HÓA cách đọc
- Viết bằng chữ cái tiếng Việt
- Không phải tiếng Việt gốc

VÍ DỤ HỢP LỆ:
xa rang hê (sarange)
ô pa (oppa)
a ni ô (aniyo)
xì tai (style)
bít boi (bad boy)
cà phê (café – nếu dùng theo nghĩa vay mượn)

KHÔNG LẤY:
- Từ tiếng Anh viết đúng chính tả
- Tên riêng

====================
3️⃣ TÊN RIÊNG (ten_rieng)
====================
BAO GỒM:
- Tên người, nghệ danh: Đen Vâu, Jombie, Sơn Tùng
- Địa danh: Sài Gòn, Hà Nội, Paris
- Thương hiệu, nhóm nhạc: BTS, Blackpink, Gucci

QUY TẮC CHUNG:
- Không lặp từ
- Giữ nguyên dạng xuất hiện trong lyrics
- Mỗi nhóm là chuỗi, các từ cách nhau bằng dấu phẩy
- Chỉ trả về JSON, không giải thích, không text thừa

FORMAT OUTPUT:
{{
  "eng": "",
  "nuoc_ngoai_phien_am": "",
  "ten_rieng": ""
}}

Nếu nhóm nào KHÔNG có → để chuỗi rỗng "".

Lyrics:
{lyrics_text}
"""


    messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt}]

    # --- PHẦN SỬA LỖI QUAN TRỌNG ---
    if tokenizer.pad_token is None: tokenizer.pad_token = tokenizer.eos_token

    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        attention_mask=model_inputs.attention_mask, # <--- Fix lỗi Warning
        pad_token_id=tokenizer.eos_token_id,        # <--- Fix lỗi lặp từ
        max_new_tokens=512, temperature=0.2, do_sample=True
    )
    # ----------------------------------

    response = tokenizer.batch_decode([out[len(inp):] for inp, out in zip(model_inputs.input_ids, generated_ids)], skip_special_tokens=True)[0]
    try:
        start = response.find('{'); end = response.rfind('}') + 1
        return json.loads(response[start:end]) if start != -1 else None
    except: return None

def main_group_1():
    if 'model' not in globals(): print("❌ Chưa load model!"); return
    try: df = pd.read_csv(INPUT_FILE_PATH, encoding='utf-8')
    except: df = pd.read_csv(INPUT_FILE_PATH, encoding='utf-16')

    df.columns = df.columns.str.strip().str.lower()
    target_col = next((col for col in df.columns if 'lyrics' in col or 'content' in col), None)

    start_row = 0
    if os.path.exists(OUTPUT_FILE_PATH):
        with open(OUTPUT_FILE_PATH, 'r', encoding='utf-8-sig') as f:
            start_row = max(0, sum(1 for line in f) - 1)
        print(f"[*] Tiếp tục từ dòng {start_row}")

    with open(OUTPUT_FILE_PATH, mode='a', encoding='utf-8-sig', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=['index_goc', 'lyrics', 'eng', 'nuoc_ngoai_phien_am', 'ten_rieng'])
        if start_row == 0: writer.writeheader()

        for i in range(start_row, len(df)):
            row = df.iloc[i]; lyrics = str(row[target_col])
            print(f"[*] Dòng {i} (Group 1)...", end="", flush=True)

            if len(lyrics) < 3 or lyrics.lower() == 'nan':
                writer.writerow({'index_goc': i, 'lyrics': 'SKIP', 'eng': '', 'nuoc_ngoai_phien_am': '', 'ten_rieng': ''})
                print(" -> Bỏ qua"); continue

            res = ask_group_1(lyrics)
            if res:
                writer.writerow({'index_goc': i, 'lyrics': lyrics,
                                 'eng': str(res.get('eng', '')),
                                 'nuoc_ngoai_phien_am': str(res.get('nuoc_ngoai_phien_am', '')),
                                 'ten_rieng': str(res.get('ten_rieng', ''))})
                f.flush(); print(" ✅")
            else:
                writer.writerow({'index_goc': i, 'lyrics': lyrics, 'eng': '', 'nuoc_ngoai_phien_am': '', 'ten_rieng': ''})
                print(" ❌")
            time.sleep(0.05)

if __name__ == "__main__": main_group_1()

[*] Tiếp tục từ dòng 59
[*] Dòng 59 (Group 1)... ✅
[*] Dòng 60 (Group 1)... ✅
[*] Dòng 61 (Group 1)... ✅
[*] Dòng 62 (Group 1)... ✅
[*] Dòng 63 (Group 1)... ✅
[*] Dòng 64 (Group 1)... ❌
[*] Dòng 65 (Group 1)... ✅
[*] Dòng 66 (Group 1)... ✅
[*] Dòng 67 (Group 1)... ✅
[*] Dòng 68 (Group 1)... ✅
[*] Dòng 69 (Group 1)...

KeyboardInterrupt: 